In [5]:
from sklearn.metrics import adjusted_rand_score
import warnings
from pandas.core.common import SettingWithCopyWarning
import pandas as pd

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [6]:
train = pd.read_csv('wiki-wiki/train.csv', sep='\t')

# Based on frequency (WSD)

In [9]:
train['dummy_pred'] = [1] * len(train)

In [10]:
len(train[train['gold_sense_id'] == train['dummy_pred']]) / len(train)

0.6355353075170843

In [11]:
adjusted_rand_score(train['dummy_pred'], train['gold_sense_id'])

0.0

# Word2Vec + Birch baseline (WSI)

In [12]:
train = train[train['context'] == train['context']]

In [16]:
import spacy
from sklearn.cluster import KMeans
from sklearn.cluster import Birch
from sklearn.cluster import AgglomerativeClustering
from gensim.models import KeyedVectors

def return_vec(string):
    vec = [0.0] * 300
    length = 0
    doc = nlp(string)
    for token in doc:
        try:
            if not token.is_punct and not token.is_quote and not token.is_stop\
                and not token.is_digit and not token.is_bracket and not token.is_title\
                and not token.is_space:
                vec += wv[token.lemma_ + '_' + token.pos_]
                length += 1
        except KeyError as E:
            continue
    return vec / length

nlp = spacy.load('ru_core_news_sm')
wv = KeyedVectors.load_word2vec_format('model_wiki.bin', binary=True)

TypeError: Plain typing.NoReturn is not valid as type argument

In [2015]:
result = pd.DataFrame()

for word in train['word'].unique():
    part = train[train['word'] == word]
    part['vector'] = part['context'].apply(return_vec)
    birch = Birch(n_clusters=2).fit(list(part['vector']))
    part['cluster'] = birch.labels_
    result = result.append(part)

In [1]:
adjusted_rand_score(result['cluster'], result['gold_sense_id'])

NameError: name 'adjusted_rand_score' is not defined

# Jamsic method (WSI)

In [1872]:
wv = KeyedVectors.load_word2vec_format('model_wiki.bin', binary=True)

In [1987]:
train = pd.read_csv('wiki-wiki/train.csv', sep='\t')

train['vector'] = train['context'].apply(return_vec)

In [2010]:
from scipy.spatial.distance import cosine
from pymorphy2 import MorphAnalyzer
from functools import lru_cache

morph = MorphAnalyzer()

def return_vecs(word):
    lemma = ms.lemmatize(word)[0]
    lst_1 = [w[0] for w in wv.similar_by_word(lemma + '_NOUN')][:1]
    total_vec_1 = [0] * 300
    for w in lst_1:
        total_vec_1 += wv[w]
    first_sense_vec = total_vec_1 / len(lst_1)
    
    lst_2 = [w[0] for w in wv.similar_by_vector((wv[lemma + '_NOUN'] - first_sense_vec))][:]
    total_vec_2 = [0] * 300
    for w in lst_2:
        total_vec_2 += wv[w]
    second_sense_vec = total_vec_2 / len(lst_2)
    return first_sense_vec, second_sense_vec

def compare_vecs(vector, first_sense_vec, second_sense_vec):
    first_sense_cosine = 1 - cosine(vector, first_sense_vec)
    second_sense_cosine = 1 - cosine(vector, second_sense_vec)
    if first_sense_cosine > second_sense_cosine:
        return 1
    else:
        return 0

In [2011]:
result = pd.DataFrame()
for word in train['word'].unique():
    part = train[train['word'] == word]
    fsv, ssv = return_vecs(word)
    part['cl'] = part['vector'].apply(compare_vecs, first_sense_vec=fsv, second_sense_vec=ssv)
    result = result.append(part)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [2012]:
adjusted_rand_score(result['cl'], result['gold_sense_id'])

0.5145054982006625